In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd

In [2]:
# Import the .txt-file generated by 'autotest.sh'
txtname = "tree_test.txt"
data = np.loadtxt(txtname)
data.shape

(24, 7)

In [3]:
def convert(x):
    if x == 0:
        return "Mono"
    else:
        return "Quad"

In [4]:
N      = "N"
theta  = "$\theta_d$"
quad   = "Multipole expansion"
tree   = "Tree code t(s)"
dirsum = "Direct t(s)"
relerr = "Relative error"
intact = "Avg interactions"

data_ls = []
for i in range(len(data)):
    data_ls.append({
        N:      int(data[i,0]),           # Number of particles
        theta:  data[i,1],                # Opening angle threshold
        quad:   convert(int(data[i,2])),  # Using quadrupoles (bool)
        tree:   data[i,3],                # Execution time tree code (sec)
        dirsum: data[i,4],                # Execution time direct summation (sec)
        relerr: data[i,5],                # Mean relative force error
        intact: data[i,6]                 # Mean number of particle-node interactions
    })
df = pd.DataFrame(data_ls)

In [5]:
df = df.groupby([N, theta, quad]).first().unstack()
df

Tree code t(s)            Direct t(s)             \
Multipole expansion           Mono       Quad        Mono       Quad   
N     $\theta_d$                                                       
5000  0.2                 0.604802   1.066630     1.78396    1.77675   
      0.4                 0.163367   0.297304     1.77674    1.77734   
      0.8                 0.034382   0.064255     1.78090    1.78542   
10000 0.2                 2.109320   3.418570     7.51979    8.09434   
      0.4                 0.485566   0.951817     8.28296    8.40635   
      0.8                 0.094773   0.184383     8.13719    8.12671   
20000 0.2                 5.444850   9.589140    31.64360   31.52720   
      0.4                 1.129170   1.927480    29.20930   29.28240   
      0.8                 0.225642   0.420236    30.22290   29.02400   
40000 0.2                16.810400  25.686300   116.58500  117.13400   
      0.4                 3.272910   4.933170   117.49900  119.81100   
      0.8                 0.587553   0.888337   117.86300  118.21700   

                    Relative error           Avg interactions            
Multipole expansion           Mono      Quad             Mono      Quad  
N     $\theta_d$                                                         
5000  0.2                 0.000279  0.000022         1735.480  1735.480  
      0.4                 0.002177  0.000384          501.845   501.845  
      0.8                 0.014926  0.006886          109.968   109.968  
10000 0.2                 0.000245  0.000020         2403.890  2403.890  
      0.4                 0.001814  0.000332          621.909   621.909  
      0.8                 0.012776  0.006108          127.757   127.757  
20000 0.2                 0.000225  0.000019         3120.220  3120.220  
      0.4                 0.001527  0.000294          737.402   737.402  
      0.8                 0.010606  0.005340          144.197   144.197  
40000 0.2                 0.000195  0.000016         3976.430  3976.430  
      0.4                 0.001299  0.000248          866.969   866.969  
      0.8                 0.008915  0.004823          161.829   161.829

Need to fix the direct summation column, misleading to have 'Mono' and 'Quad' in its current position. We calculate the mean execution time of the direct summations at each row.

In [6]:
# Calculate the mean of direct summation columns, insert into the dataframe.
df[('Direct t(s)','-')] = df.iloc[:,2:4].mean(axis=1)

# Drop the two old direct summation columns.
df = df.drop(df.columns[2:4], axis=1)

# Get list of column names.
col_list = list(df.columns)

# Insert mean of direct summation into old position in dataframe.
col_list.insert(2,col_list[-1])
col_list.pop(-1)
df = df[col_list]

In [7]:
# Need to fix formatting of floating points number. REMOVE THIS SHIT
label_list = list(df.columns)
df.style.format({
    label_list[0]: "{:.3f}", # Tree, mono
    label_list[1]: '{:.3f}', # Tree, quad
    label_list[2]: '{:.3f}', # Direct
    label_list[3]: '{:.2e}', # Rel err, mono
    label_list[4]: '{:.2e}', # Rel err, quad
    label_list[5]: '{:.1f}', # Avg int, mono
    label_list[6]: '{:.1f}', # Avg int, quad
})

In [8]:
format_1 = "{0:.3f}".format
format_2 = "{0:.2e}".format
format_3 = "{0:.1f}".format
df[list(df.columns)[0:3]]= df[list(df.columns)[0:3]].applymap(format_1)
df[list(df.columns)[3:5]]= df[list(df.columns)[3:5]].applymap(format_2)
df[list(df.columns)[5:]] = df[list(df.columns)[5:]].applymap(format_3)
df

Tree code t(s)         Direct t(s) Relative error  \
Multipole expansion           Mono    Quad           -           Mono   
N     $\theta_d$                                                        
5000  0.2                    0.605   1.067       1.780       2.79e-04   
      0.4                    0.163   0.297       1.777       2.18e-03   
      0.8                    0.034   0.064       1.783       1.49e-02   
10000 0.2                    2.109   3.419       7.807       2.45e-04   
      0.4                    0.486   0.952       8.345       1.81e-03   
      0.8                    0.095   0.184       8.132       1.28e-02   
20000 0.2                    5.445   9.589      31.585       2.25e-04   
      0.4                    1.129   1.927      29.246       1.53e-03   
      0.8                    0.226   0.420      29.623       1.06e-02   
40000 0.2                   16.810  25.686     116.859       1.95e-04   
      0.4                    3.273   4.933     118.655       1.30e-03   
      0.8                    0.588   0.888     118.040       8.91e-03   

                              Avg interactions          
Multipole expansion      Quad             Mono    Quad  
N     $\theta_d$                                        
5000  0.2            2.21e-05           1735.5  1735.5  
      0.4            3.84e-04            501.8   501.8  
      0.8            6.89e-03            110.0   110.0  
10000 0.2            2.01e-05           2403.9  2403.9  
      0.4            3.32e-04            621.9   621.9  
      0.8            6.11e-03            127.8   127.8  
20000 0.2            1.90e-05           3120.2  3120.2  
      0.4            2.94e-04            737.4   737.4  
      0.8            5.34e-03            144.2   144.2  
40000 0.2            1.59e-05           3976.4  3976.4  
      0.4            2.48e-04            867.0   867.0  
      0.8            4.82e-03            161.8   161.8

In [10]:
print(df.to_latex())

\begin{tabular}{lllllllll}
\toprule
      & {} & \multicolumn{2}{l}{Tree code t(s)} & Direct t(s) & \multicolumn{2}{l}{Relative error} & \multicolumn{2}{l}{Avg interactions} \\
      & Multipole expansion &           Mono &    Quad &           - &           Mono &      Quad &             Mono &    Quad \\
N & \$\textbackslash theta\_d\$ &                &         &             &                &           &                  &         \\
\midrule
5000  & 0.2 &          0.605 &   1.067 &       1.780 &       2.79e-04 &  2.21e-05 &           1735.5 &  1735.5 \\
      & 0.4 &          0.163 &   0.297 &       1.777 &       2.18e-03 &  3.84e-04 &            501.8 &   501.8 \\
      & 0.8 &          0.034 &   0.064 &       1.783 &       1.49e-02 &  6.89e-03 &            110.0 &   110.0 \\
10000 & 0.2 &          2.109 &   3.419 &       7.807 &       2.45e-04 &  2.01e-05 &           2403.9 &  2403.9 \\
      & 0.4 &          0.486 &   0.952 &       8.345 &       1.81e-03 &  3.32e-04 &           